# Preliminaries

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pyspark.sql
from pyspark.sql import Row
from pyspark.sql.types import *
import json

# Import data

In [7]:
myPath = "/content/drive/My Drive/H516/"

In [8]:
rawdataDF = spark.read.csv(myPath + "spotify_songs_cleaner.csv", header="true")
rawdataDF.createOrReplaceTempView("spotify_songs")

# Preview data

In [9]:
lyricPreview = spark.sql("SELECT * FROM spotify_songs ")
lyricPreview.show(10)

+--------------------+--------------------+---------------+----------------+--------------------+--------------------+------------------------+--------------------+--------------------+--------------+------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------+--------------------+
|            track_id|          track_name|   track_artist|track_popularity|      track_album_id|    track_album_name|track_album_release_date|       playlist_name|         playlist_id|playlist_genre| playlist_subgenre|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|language|             lyrics2|
+--------------------+--------------------+---------------+----------------+--------------------+--------------------+------------------------+--------------------+--------------------+--------------+------------------+------------+------+---+--------+----+---------

In [10]:
lyricGenres = spark.sql("SELECT DISTINCT playlist_genre, COUNT(track_id) AS TTL_RCRDS, COUNT(DISTINCT track_id) AS SONG_COUNT FROM spotify_songs WHERE playlist_genre IN('pop','rap','rock','r&b','edm') AND language = 'en' GROUP BY playlist_genre ")
lyricGenres.show()

+--------------+---------+----------+
|playlist_genre|TTL_RCRDS|SONG_COUNT|
+--------------+---------+----------+
|           r&b|     3152|      3152|
|           pop|     3738|      3738|
|           edm|     1758|      1758|
|           rap|     2502|      2502|
|          rock|     3388|      3388|
+--------------+---------+----------+



# Filter data for selected genres and language (add more filters if needed)

In [11]:
genres = ['pop','rap','rock','r&b','edm']
lyricsR = rawdataDF.rdd.filter(lambda r: r.playlist_genre in genres and r.language == 'en' and r.track_name != 'Dumb Litty' and r.track_album_release_date[-3] != '-').map(lambda r: (r.track_id,r.playlist_genre,r.track_popularity,r.track_album_release_date[-4:],round(float(r.duration_ms)/60000,1),r.lyrics2))
lyricsR.take(3)

[('004s3t0ONYlzxII9PLgU6z',
  'rock',
  '28',
  '2017',
  6.2,
  '[The trees, are singing in the wind The sky blue, only as it can be And the angels, smiled at me I saw you, in that lonely bench At half past four, I kissed your soft soft hands and at 6 I kissed your lips and the angels smiled, I thought Hey I feel alive! The park sign, said it was closed And we jumped that fence with no cares at all and we kissed under a tree We danced, under the midnight sun And I loved you, without knowing you at all and we laughed and felt so free and the angels they smiled, I thought Hey, I feel alive!]'),
 ('00chLpzhgVjxs1zKC9UScL',
  'r&b',
  '0',
  '2005',
  4.4,
  '"[NA Yeah, Spyderman and Freeze in full effect Uh-huh You ready, Ron? I\'m ready You ready, Biv? I\'m ready, Slick, are you? Oh, yeah, break it down NA Girl, I, must (warn you) I sense something strange in my mind Situation is (serious) Let\'s cure it cause we\'re running out of time It\'s oh, so (beautiful) Relationships they seem f

# Remove punctuation and numbers from lyrics

> Indented block



In [12]:
import string

In [13]:
#remove punctuation
wordsR = lyricsR.map(lambda x:(x[0],x[1],x[2],x[3],x[4],x[5].translate(str.maketrans('','',string.punctuation))))
wordsR.take(2)

[('004s3t0ONYlzxII9PLgU6z',
  'rock',
  '28',
  '2017',
  6.2,
  'The trees are singing in the wind The sky blue only as it can be And the angels smiled at me I saw you in that lonely bench At half past four I kissed your soft soft hands and at 6 I kissed your lips and the angels smiled I thought Hey I feel alive The park sign said it was closed And we jumped that fence with no cares at all and we kissed under a tree We danced under the midnight sun And I loved you without knowing you at all and we laughed and felt so free and the angels they smiled I thought Hey I feel alive'),
 ('00chLpzhgVjxs1zKC9UScL',
  'r&b',
  '0',
  '2005',
  4.4,
  'NA Yeah Spyderman and Freeze in full effect Uhhuh You ready Ron Im ready You ready Biv Im ready Slick are you Oh yeah break it down NA Girl I must warn you I sense something strange in my mind Situation is serious Lets cure it cause were running out of time Its oh so beautiful Relationships they seem from the start Its all so deadly When love is no

In [14]:
#remove numbers
wordsR2 = wordsR.map(lambda x:(x[0],x[1],x[2],x[3],x[4],x[5].translate(str.maketrans('','',string.digits))))
wordsR2.take(2)

[('004s3t0ONYlzxII9PLgU6z',
  'rock',
  '28',
  '2017',
  6.2,
  'The trees are singing in the wind The sky blue only as it can be And the angels smiled at me I saw you in that lonely bench At half past four I kissed your soft soft hands and at  I kissed your lips and the angels smiled I thought Hey I feel alive The park sign said it was closed And we jumped that fence with no cares at all and we kissed under a tree We danced under the midnight sun And I loved you without knowing you at all and we laughed and felt so free and the angels they smiled I thought Hey I feel alive'),
 ('00chLpzhgVjxs1zKC9UScL',
  'r&b',
  '0',
  '2005',
  4.4,
  'NA Yeah Spyderman and Freeze in full effect Uhhuh You ready Ron Im ready You ready Biv Im ready Slick are you Oh yeah break it down NA Girl I must warn you I sense something strange in my mind Situation is serious Lets cure it cause were running out of time Its oh so beautiful Relationships they seem from the start Its all so deadly When love is not

# Add the total number of words, and the number of stopwords and profane words

In [15]:
!pip install profanity


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for profanity: filename=profanity-1.1-py3-none-any.whl size=4253 sha256=80330558ac253300f586f48d9f2761b30643e77fc7f4e4f1c4ad04aa218b07a1
  Stored in directory: /root/.cache/pip/wheels/d6/62/ce/2d1388b4998148d18bc3fd287105fe25f63ed695e287928ef1
Successfully built profanity


In [16]:
from profanity import profanity

In [17]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
english_stopwprds = stopwords.words('english')

In [20]:
#use the positive word list from assignment 4
poswordsR = sc.textFile(myPath + "pos.txt")
poswordsR.take(5)

['abound', 'abounds', 'abundance', 'abundant', 'accessable']

In [21]:
#use the negative word list from assignment 4
negwordsR = sc.textFile(myPath + "neg.txt")
negwordsR.take(5)

['abnormal', 'abolish', 'abominable', 'abominably', 'abominate']

In [22]:
#get python lists of positive an negative words from input files
poswords = poswordsR.collect()
negwords = negwordsR.collect()

In [23]:
#create a function to calculate the number of words in the song
getLength = lambda x: len(x.split())

In [24]:
#create a function to calculate the number of stop words in the song
getStopCount = lambda x: len([x.split()[stopword] for stopword in range(len(x.split())) if x.split()[stopword] in english_stopwprds])

In [25]:
#create a function to calculate the number of profane words in the song
getProfaneCount = lambda x: len([x.split()[badword] for badword in range(len(x.split())) if profanity.contains_profanity(x.split()[badword]) == True])

In [26]:
#create a function to measure the amount of repetition in a song
getUniqueCount = lambda x: len(set([x.split()[unique] for unique in range(len(x.split())) if x.split()[unique] not in english_stopwprds]))

In [27]:
#create a function to calculate the number of positive words in the song
getPositiveCount = lambda x: len([x.split()[posword] for posword in range(len(x.split())) if x.split()[posword] in poswords])

In [28]:
#create a function to calculate the number of positive words in the song
getNegativeCount = lambda x: len([x.split()[negword] for negword in range(len(x.split())) if x.split()[negword] in negwords])

In [29]:
extrColsR = wordsR.map(lambda x: (x[0],x[1],x[2],int(x[3]),x[4],getLength(x[5]),int(getLength(x[5])/x[4]), getUniqueCount(x[5]), round((1-getUniqueCount(x[5])/getLength(x[5])),2), getStopCount(x[5]),round((getStopCount(x[5])/getLength(x[5])),2), getProfaneCount(x[5]),round((getProfaneCount(x[5])/getLength(x[5])),2), getPositiveCount(x[5]),round((getPositiveCount(x[5])/getLength(x[5])),2), getNegativeCount(x[5]),round((getNegativeCount(x[5])/getLength(x[5])),2), x[5]))
extrColsR.take(2)

[('004s3t0ONYlzxII9PLgU6z',
  'rock',
  '28',
  2017,
  6.2,
  111,
  17,
  44,
  0.6,
  45,
  0.41,
  0,
  0.0,
  4,
  0.04,
  1,
  0.01,
  'The trees are singing in the wind The sky blue only as it can be And the angels smiled at me I saw you in that lonely bench At half past four I kissed your soft soft hands and at 6 I kissed your lips and the angels smiled I thought Hey I feel alive The park sign said it was closed And we jumped that fence with no cares at all and we kissed under a tree We danced under the midnight sun And I loved you without knowing you at all and we laughed and felt so free and the angels they smiled I thought Hey I feel alive'),
 ('00chLpzhgVjxs1zKC9UScL',
  'r&b',
  '0',
  2005,
  4.4,
  129,
  29,
  67,
  0.48,
  45,
  0.35,
  0,
  0.0,
  10,
  0.08,
  4,
  0.03,
  'NA Yeah Spyderman and Freeze in full effect Uhhuh You ready Ron Im ready You ready Biv Im ready Slick are you Oh yeah break it down NA Girl I must warn you I sense something strange in my mind Sit

In [30]:
labeledR = extrColsR.map(lambda p: Row(track_id=p[0],genre=p[1],popularity=p[2],year=p[3],minutes=p[4], word_count=p[5], words_per_minute=p[6],unique_words=p[7], repetition_pct=p[8],stopword_count=p[9],stopword_pct=p[10],profanity_count=p[11],profanity_pct=p[12],positive_count=p[13],positive_pct=p[14],negative_count=p[15],negative_pct=p[16], lyrics=p[17]))
labeledR.take(3)

[Row(track_id='004s3t0ONYlzxII9PLgU6z', genre='rock', popularity='28', year=2017, minutes=6.2, word_count=111, words_per_minute=17, unique_words=44, repetition_pct=0.6, stopword_count=45, stopword_pct=0.41, profanity_count=0, profanity_pct=0.0, positive_count=4, positive_pct=0.04, negative_count=1, negative_pct=0.01, lyrics='The trees are singing in the wind The sky blue only as it can be And the angels smiled at me I saw you in that lonely bench At half past four I kissed your soft soft hands and at 6 I kissed your lips and the angels smiled I thought Hey I feel alive The park sign said it was closed And we jumped that fence with no cares at all and we kissed under a tree We danced under the midnight sun And I loved you without knowing you at all and we laughed and felt so free and the angels they smiled I thought Hey I feel alive'),
 Row(track_id='00chLpzhgVjxs1zKC9UScL', genre='r&b', popularity='0', year=2005, minutes=4.4, word_count=129, words_per_minute=29, unique_words=67, repeti

In [31]:
%%time
labeledDF = spark.createDataFrame(labeledR)
labeledDF.show(5)

+--------------------+-----+----------+----+-------+----------+----------------+------------+--------------+--------------+------------+---------------+-------------+--------------+------------+--------------+------------+--------------------+
|            track_id|genre|popularity|year|minutes|word_count|words_per_minute|unique_words|repetition_pct|stopword_count|stopword_pct|profanity_count|profanity_pct|positive_count|positive_pct|negative_count|negative_pct|              lyrics|
+--------------------+-----+----------+----+-------+----------+----------------+------------+--------------+--------------+------------+---------------+-------------+--------------+------------+--------------+------------+--------------------+
|004s3t0ONYlzxII9P...| rock|        28|2017|    6.2|       111|              17|          44|           0.6|            45|        0.41|              0|          0.0|             4|        0.04|             1|        0.01|The trees are sin...|
|00chLpzhgVjxs1zKC...|  

# Save the data to a file so that it will not have to be recalculated every time

In [37]:
#save the enhanced data frame to a csv file so that we will not have to recalulate it every time
%%time
labeledDF.write.mode('overwrite').options(header='True', delimiter='~').csv(myPath + "more_lyric_columns.csv")

#Read the file in again to do more analysis without recreating the data every time

In [41]:
from pyspark.sql.types import StructField, StructType, StringType,LongType

In [43]:
custom_schema = StructType([
    StructField("track_id", StringType(), False),
    StructField("genre", StringType(), False),
    StructField("popularity", IntegerType(), False),
    StructField("year", IntegerType(), False),
    StructField("minutes", FloatType(), False),
    StructField("word_count", IntegerType(), False),
    StructField("words_per_minute", FloatType(), False),
    StructField("unique_words", IntegerType(), False),
    StructField("repetition_pct", FloatType(), False),
    StructField("stopword_count", IntegerType(), False),
    StructField("stopword_pct", FloatType(), False),
    StructField("profanity_count", IntegerType(), False),
    StructField("profanity_pct", FloatType(), False),
    StructField("positive_count", IntegerType(), False),
    StructField("positive_pct", FloatType(), False),
    StructField("negative_count", IntegerType(), False),
    StructField("negative_pct", FloatType(), False),
    StructField("words_only_lyrics", StringType(),False)
])

In [47]:
newDF = spark.read.schema(custom_schema).option("header", True).option("delimiter", "~").csv(myPath + "more_lyric_columns.csv")
newDF.show(5)

+--------------------+-----+----------+----+-------+----------+----------------+------------+--------------+--------------+------------+---------------+-------------+--------------+------------+--------------+------------+--------------------+
|            track_id|genre|popularity|year|minutes|word_count|words_per_minute|unique_words|repetition_pct|stopword_count|stopword_pct|profanity_count|profanity_pct|positive_count|positive_pct|negative_count|negative_pct|   words_only_lyrics|
+--------------------+-----+----------+----+-------+----------+----------------+------------+--------------+--------------+------------+---------------+-------------+--------------+------------+--------------+------------+--------------------+
|004s3t0ONYlzxII9P...| rock|        28|2017|    6.2|       111|            17.0|          44|           0.6|            45|        0.41|              0|          0.0|             4|        0.04|             1|        0.01|The trees are sin...|
|00chLpzhgVjxs1zKC...|  

In [48]:
#Convert genres to numberic values
classes = {'rock':0,'pop':1,'r&b':2,'rap':3,'edm':4}

In [74]:
#reset the fields to numeric values and remove old years (since 50s and 60s songs were much different than songs starting in the 70s)
newR = newDF.rdd.map(lambda x: (x[0],x[1],classes[x[1]],int(x[2]),int(int(x[3])/10)*10,round(float(x[4]),1), int(x[5]), int(x[6]),int(x[7]),round(float(x[8]),3),round(float(x[10]),3),int(x[11]),round(float(x[12]),4),int(x[13]),round(float(x[14]),3),int(x[15]),round(float(x[16]),3),x[17])).filter(lambda x: x[4] >= 1970)
newR.take(2)

[('004s3t0ONYlzxII9PLgU6z',
  'rock',
  0,
  28,
  2010,
  6.2,
  111,
  17,
  44,
  0.6,
  0.41,
  0,
  0.0,
  4,
  0.04,
  1,
  0.01,
  'The trees are singing in the wind The sky blue only as it can be And the angels smiled at me I saw you in that lonely bench At half past four I kissed your soft soft hands and at 6 I kissed your lips and the angels smiled I thought Hey I feel alive The park sign said it was closed And we jumped that fence with no cares at all and we kissed under a tree We danced under the midnight sun And I loved you without knowing you at all and we laughed and felt so free and the angels they smiled I thought Hey I feel alive'),
 ('00chLpzhgVjxs1zKC9UScL',
  'r&b',
  2,
  0,
  2000,
  4.4,
  129,
  29,
  67,
  0.48,
  0.35,
  0,
  0.0,
  10,
  0.08,
  4,
  0.03,
  'NA Yeah Spyderman and Freeze in full effect Uhhuh You ready Ron Im ready You ready Biv Im ready Slick are you Oh yeah break it down NA Girl I must warn you I sense something strange in my mind Situation

In [75]:
#relabel the fields
newLabeledR = newR.map(lambda p: Row(track_id=p[0], genre=p[1], genreID=p[2], popularity=p[3], year=p[4], minutes=p[5], word_count=p[6], words_per_minute=p[7],unique_words=p[8], repetition_pct=p[9], stopword_pct=p[10], profanity_count=p[11], profanity_pct=p[12], positive_count=p[13], positive_pct=p[14], negative_count=p[15], negative_pct=p[16], lyrics=p[17]))
newLabeledR.take(2)

[Row(track_id='004s3t0ONYlzxII9PLgU6z', genre='rock', genreID=0, popularity=28, year=2010, minutes=6.2, word_count=111, words_per_minute=17, unique_words=44, repetition_pct=0.6, stopword_pct=0.41, profanity_count=0, profanity_pct=0.0, positive_count=4, positive_pct=0.04, negative_count=1, negative_pct=0.01, lyrics='The trees are singing in the wind The sky blue only as it can be And the angels smiled at me I saw you in that lonely bench At half past four I kissed your soft soft hands and at 6 I kissed your lips and the angels smiled I thought Hey I feel alive The park sign said it was closed And we jumped that fence with no cares at all and we kissed under a tree We danced under the midnight sun And I loved you without knowing you at all and we laughed and felt so free and the angels they smiled I thought Hey I feel alive'),
 Row(track_id='00chLpzhgVjxs1zKC9UScL', genre='r&b', genreID=2, popularity=0, year=2000, minutes=4.4, word_count=129, words_per_minute=29, unique_words=67, repetit

In [76]:
newLabeledDF = spark.createDataFrame(newLabeledR)
newLabeledDF.createOrReplaceTempView("lyric_features")

# Review the new data set

In [64]:
checkYears = spark.sql("SELECT DISTINCT year FROM lyric_features ORDER BY year")
checkYears.show(100)

+----+
|year|
+----+
|1970|
|1980|
|1990|
|2000|
|2010|
|2020|
+----+



In [79]:
Review = spark.sql("SELECT * FROM lyric_features ORDER BY year")
Review.show(3)

+--------------------+-----+-------+----------+----+-------+----------+----------------+------------+--------------+------------+---------------+-------------+--------------+------------+--------------+------------+--------------------+
|            track_id|genre|genreID|popularity|year|minutes|word_count|words_per_minute|unique_words|repetition_pct|stopword_pct|profanity_count|profanity_pct|positive_count|positive_pct|negative_count|negative_pct|              lyrics|
+--------------------+-----+-------+----------+----+-------+----------+----------------+------------+--------------+------------+---------------+-------------+--------------+------------+--------------+------------+--------------------+
|4GetBW8bJ8lB2Y9ug...| rock|      0|        26|1970|    4.8|        99|              20|          52|          0.47|        0.36|              0|          0.0|             2|        0.02|             3|        0.03|Oooh yeah Get you...|
|4gMgiXfqyzZLMhsks...| rock|      0|        76|1970|

In [68]:
#Review the averages of all the new columns by genre
%%time
avgs = spark.sql("SELECT genre, round(SUM(popularity)/COUNT(1),0) AS avg_popularity, round(SUM(minutes)/COUNT(1),1) AS avg_minutes, round(SUM(word_count)/COUNT(1),0) AS avg_word_count, round(SUM(words_per_minute)/COUNT(1),0) AS avg_words_per_minute, round(SUM(repetition_pct)/COUNT(1),2)  AS avg_repetition_pct, round(SUM(stopword_pct)/COUNT(1),2) AS avg_stopword_pct, round(SUM(profanity_count)/COUNT(1),2) as avg_profanity_count, round(SUM(profanity_pct)/COUNT(1),3) as avg_profanity_pct,  round(SUM(negative_count)/COUNT(1),0) AS avg_negative_count, round(SUM(negative_pct)/COUNT(1),2) AS avg_negative_pct, round(SUM(positive_count)/COUNT(1),0) as avg_positive_count, round(SUM(positive_pct)/COUNT(1),2) as avg_positive_pct FROM lyric_features GROUP BY genre ORDER BY 5")
avgs.show()

+-----+--------------+-----------+--------------+--------------------+------------------+----------------+-------------------+-----------------+------------------+----------------+------------------+----------------+
|genre|avg_popularity|avg_minutes|avg_word_count|avg_words_per_minute|avg_repetition_pct|avg_stopword_pct|avg_profanity_count|avg_profanity_pct|avg_negative_count|avg_negative_pct|avg_positive_count|avg_positive_pct|
+-----+--------------+-----------+--------------+--------------------+------------------+----------------+-------------------+-----------------+------------------+----------------+------------------+----------------+
| rock|          42.0|        4.2|         250.0|                62.0|              0.69|            0.36|               0.33|            0.001|               9.0|            0.04|               8.0|            0.03|
|  edm|          33.0|        3.7|         255.0|                72.0|              0.76|            0.36|               0.64|      

# Join the new data with the original data to combine all fields

In [80]:
doubleDataDF = spark.sql("SELECT a.track_id, a.track_name, a.track_artist, a.track_popularity, a.track_album_id, a.track_album_name, a.track_album_release_date, a.playlist_name, a.playlist_id, a.playlist_genre, a.playlist_subgenre, a.danceability, a.energy, a.key, a.loudness, a.mode, a.speechiness, a.acousticness, a.instrumentalness, a.liveness, a.valence, a.tempo, a.duration_ms, a.language, b.genreID, b.year, b.minutes, b.word_count, b.words_per_minute, b.repetition_pct, b.stopword_pct, b.profanity_pct, b.negative_pct, b.positive_pct, CASE WHEN b.positive_count > b.negative_count THEN 2 WHEN b.negative_count > b.positive_count THEN 0 ELSE 1 END AS Sentiment , b.lyrics AS words_only_lyrics FROM spotify_songs a JOIN lyric_features b ON a.track_id = b.track_id ")
doubleDataDF.show(10)

+--------------------+--------------------+---------------+----------------+--------------------+--------------------+------------------------+--------------------+--------------------+--------------+------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------+-------+----+-------+----------+----------------+--------------+------------+-------------+------------+------------+---------+--------------------+
|            track_id|          track_name|   track_artist|track_popularity|      track_album_id|    track_album_name|track_album_release_date|       playlist_name|         playlist_id|playlist_genre| playlist_subgenre|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|language|genreID|year|minutes|word_count|words_per_minute|repetition_pct|stopword_pct|profanity_pct|negative_pct|positive_pct|Sentiment|   words_only_lyrics|
+-----------

In [71]:
doubleDataDF.createOrReplaceTempView("all_features")

In [73]:
#Review the averages of all the new columns by genre
%%time
moreAvgs = spark.sql("SELECT playlist_genre, round(SUM(danceability)/COUNT(1),2) AS avg_danceability, round(SUM(energy)/COUNT(1),2) AS avg_energy, round(SUM(loudness)/COUNT(1),2) AS avg_loudness, round(SUM(speechiness)/COUNT(1),2) AS avg_speechiness, round(SUM(acousticness)/COUNT(1),2)  AS avg_acousticness, round(SUM(instrumentalness)/COUNT(1),2) AS avg_instrumentalness, round(SUM(liveness)/COUNT(1),2) as avg_liveness, round(SUM(valence)/COUNT(1),2) AS avg_valence, round(SUM(tempo)/COUNT(1),0) as avg_tempo FROM all_features GROUP BY playlist_genre")
moreAvgs.show()

+--------------+----------------+----------+------------+---------------+----------------+--------------------+------------+-----------+---------+
|playlist_genre|avg_danceability|avg_energy|avg_loudness|avg_speechiness|avg_acousticness|avg_instrumentalness|avg_liveness|avg_valence|avg_tempo|
+--------------+----------------+----------+------------+---------------+----------------+--------------------+------------+-----------+---------+
|           r&b|            0.67|      0.58|       -8.01|           0.11|            0.26|                0.02|        0.17|       0.54|    114.0|
|           pop|            0.64|       0.7|       -6.28|           0.07|            0.17|                0.05|        0.18|       0.51|    121.0|
|           edm|            0.64|       0.8|        -5.0|           0.08|            0.08|                 0.1|        0.21|       0.41|    126.0|
|           rap|            0.72|      0.68|       -6.62|           0.21|            0.13|                0.02|       

In [86]:
#save the cobined data frame to a csv file so that we will not have to recalulate it every time
%%time
doubleDataPD = doubleDataDF.toPandas()

CPU times: user 356 ms, sys: 33.2 ms, total: 389 ms
Wall time: 886 ms


In [87]:
doubleDataPD.index = doubleDataPD.pop("track_id")
doubleDataPD.head(5)

,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,...,minutes,word_count,words_per_minute,repetition_pct,stopword_pct,profanity_pct,negative_pct,positive_pct,Sentiment,words_only_lyrics
track_id,,,,,,,,,,,,,,,,,,,,,
0EDQwboQDmswDRn58wcslg,Kingston,Faye Webster,59,4vt0V1SmkaK1Y440P5Nsb4,Atlanta Millionaires Club,5/24/2019,Ultimate Indie Presents... Best Tracks of 2019,37i9dQZF1DWTc5QDlvD7t0,r&b,hip pop,...,3.4,220,64,0.72,0.42,0.00,0.01,0.01,2,The day that I met you I started dreaming Now ...
0HaAoQJ1PgF0gIm1o6JYIM,Where Them Niggaz At?,Pastor Troy,9,5almm5i3TH8NFWo1M1iEJ1,Face Off Part II,3/1/2005,Southern Hip Hop,57sYMLFXGD4ZqizzcMD7lz,rap,southern hip hop,...,3.6,380,105,0.55,0.28,0.08,0.09,0.01,0,This that hard shit hard shit This that hard s...
0i3utbege8Iy3q7Hy86QCF,Klanga - De Hofnar Remix,Gostan,5,4rcRFEkfAeLD4GMheeTSny,Klanga,11/28/2014,Chillout & Remixes 💜,4NlAd9NpIa92IjErMyAriM,pop,indie poptimism,...,4.4,105,23,0.55,0.30,0.01,0.01,0.02,2,Two thousand years ago Two thousand years ago ...
0IXpUl1fn2QZcBavfuq0H4,Living After Midnight,Judas Priest,63,5bqtZRbUZUxUps8mrO9tGY,British Steel,1980,Workday: Rock Classics,37i9dQZF1DX1lwxXvReaux,rock,classic rock,...,3.5,340,97,0.80,0.25,0.00,0.01,0.04,2,Living after midnight Rocking to the dawn Lovi...
0ooy3NjwsJreceWYCxlfQt,Never Let Me Down Again,Depeche Mode,10,5Yyx661Ksxl2pmRUuGLzw3,Music for the Masses,9/28/1987,"Maxi Pop GOLD (New Wave, Electropop, Synth Po...",2nRWtTI9a2LWjJ9Wy3JZs5,pop,electropop,...,4.8,231,48,0.81,0.38,0.00,0.00,0.02,2,Im taking a ride with my best friend I hope he...


In [90]:
#save the cobined data frame to a csv file so that we will not have to recalulate it every time
doubleDataPD.to_csv(myPath + "spotify_with_word_counts.csv")
